This notebook demonstrates implementing small-grid dithers (SGDs) for NIRCam in Pandeia. By default, the Pandeia engine relies on a library of sparsely-sampled, pre-computed PSFs which are spatially interpolated to produce the PSF at any given point. SGDs are an attempt to introduce PSF diversity by sampling variations that arise with small-scale offsets behind the coronagraph masks, so relying on this pre-computed library may not be appropriate. We therefore demonstrate SGDs with this library and by computing PSFs on-the-fly with WebbPSF.

This notebook assumes you've already look through the miri/nircam_photon_noise_and_contrast.ipynb notebooks, which demonstrate more basic functionality.

In [ ]:
import matplotlib.pyplot as plt
%matplotlib inline

import jwst_pancake as pancake

from pandeia.engine.calc_utils import build_default_calc
from pandeia.engine.perform_calculation import perform_calculation

from copy import deepcopy
import numpy as np

# Scene Construction

Here, we'll simply read in a template with predefined targets and planets.

In [ ]:
# Load the template
config = build_default_calc('jwst', 'nircam', 'coronagraphy')

# Set up basic parameters
target = deepcopy(config['scene'][0])
target['spectrum']['sed'] = {'sed_type': 'phoenix', 'key': 'b3v'}
# target['spectrum']['normalization'] = {'type': 'photsys', 'norm_fluxunit': 'abmag'}
target['spectrum']['normalization'] = {'type': 'jwst', 'norm_fluxunit': 'abmag'}
reference = config['strategy']['psf_subtraction_source']
reference['spectrum']['sed'] = {'sed_type': 'phoenix', 'key': 'b1v'}
# reference['spectrum']['normalization'] = {'type': 'photsys', 'norm_fluxunit': 'abmag'}
reference['spectrum']['normalization'] = {'type': 'jwst', 'norm_fluxunit': 'abmag'}

# Set up a basic planet
planet_a = deepcopy(target)
planet_a['spectrum']['sed'] = {'sed_type': 'blackbody'}
planet_a['spectrum']['normalization'] = {'type': 'jwst', 'norm_fluxunit': 'abmag'}

# Copy it to make a second planet
planet_b = deepcopy(planet_a)

# Set up Scene Parameters
# target_bandpass = 'johnson,j'
target_bandpass = 'miri,imaging,f560w'
target_abmag = 12.0

# reference_bandpass = 'johnson,j'
reference_bandpass = 'miri,imaging,f560w'
reference_abmag = 10.8

planet_a_position = {'x_offset': 0.45, 'y_offset': -0.61}
planet_a_bandpass = 'miri,imaging,f560w'
planet_a_abmag = 15.8
planet_a_temp = 900

planet_b_position = {'x_offset': -0.75, 'y_offset': 1.02}
planet_b_bandpass = 'miri,imaging,f560w'
planet_b_abmag = 16.6
planet_b_temp = 800

# Set configuration to use scene parameters
target['id'] = 1
target['spectrum']['normalization']['bandpass'] = target_bandpass
target['spectrum']['normalization']['norm_flux'] = target_abmag
config['scene'] = [target]

reference['spectrum']['normalization']['bandpass'] = reference_bandpass
reference['spectrum']['normalization']['norm_flux'] = reference_abmag
reference['id'] = 99

planet_a['id'] = 2
planet_a['spectrum']['sed']['temp'] = planet_a_temp
planet_a['spectrum']['normalization']['bandpass'] = planet_a_bandpass
planet_a['spectrum']['normalization']['norm_flux'] = planet_a_abmag
planet_a['position'] = planet_a_position
config['scene'].append(planet_a)

planet_b['id'] = 3
planet_b['spectrum']['sed']['temp'] = planet_b_temp
planet_b['spectrum']['normalization']['bandpass'] = planet_b_bandpass
planet_b['spectrum']['normalization']['norm_flux'] = planet_b_abmag
planet_b['position'] = planet_b_position
config['scene'].append(planet_b)

# Add target acquisition error
errx, erry = pancake.scene.get_ta_error()
pancake.scene.offset_scene(config['scene'], errx, erry)

# The same, but for reference
errx_ref, erry_ref = pancake.scene.get_ta_error()
pancake.scene.offset_scene([config['strategy']['psf_subtraction_source']], errx_ref, erry_ref)

To simulate a small-grid dither (SGD), pass in a reference calculation file (after adding TA error) to the ```scene.create_SGD``` function. By default, this only supports a square 9-point grid with 2 mas error on the grid points, but this function can be easily reproduced for other SGD configurations.

In [ ]:
sgds = pancake.scene.create_SGD(ta_error=False, stepsize=20.e-3)

sgd_configs = []
for s in sgds:
    current_config = deepcopy(config)
    pancake.scene.offset_scene([current_config['strategy']['psf_subtraction_source']], *s)
    sgd_configs.append(current_config)

And now we'll plot the two scenes we've constructed

In [ ]:
plt.figure(figsize=(15,5))
plt.subplot(121,polar=True)
pancake.scene.plot_scene(config['scene'],'Target Scene w/ Companion',newfig=False)
ax = plt.gca()
ax.set_rlim(0,5.)
plt.subplot(122,polar=True)
for s in sgd_configs:
    pancake.scene.plot_scene([s['strategy']['psf_subtraction_source']],'SGD Points',newfig=False)
ax = plt.gca()

# On-the-fly PSFs

To enable on-the-fly PSFs with WebbPSF, simply toggle ```engine.on_the_fly_PSFs``` to ```True``` and re-submit the calculations as before.

NB: The number of PSFs generated is given by ```engine.wave_sampling``` * number of sources + ```engine.wave_sampling``` * SGD points. For this example, we're relying on WebbPSF for 280 calculations, which can take a long time. Subsequent calculations can be sped up by use of an [LRU cache](https://en.wikipedia.org/wiki/Cache_replacement_policies#Least_Recently_Used_.28LRU.29) (enabled by default), but this caching system is incompatible with multiprocessing, so ```engine.calculate_batch``` will not contribute to the cache, while ```engine.perform_calculation``` will.

In [ ]:
options = pancake.engine.options
initial_options = options.current_options
options.wave_sampling = 21
options.on_the_fly_PSFs = True
otf_options = options.current_options

print("Calculating Target")
otf_targ_results = pancake.engine.calculate_target(config)
otf_sgd_results = []
for i, s in enumerate(sgd_configs):
    print("Calculating SGD {} of {}".format(i+1, len(sgd_configs)))
    otf_sgd_results.append(pancake.engine.calculate_reference(s))

options.current_options = initial_options

In [ ]:
target_slope = otf_targ_results['2d']['detector']

sgd_reg = []
for r in otf_sgd_results:
    slope = r['2d']['detector']
    reg = pancake.analysis.register_to_target(slope,target_slope,rescale_reference=True)
    sgd_reg.append(reg)
sgd_reg = np.array(sgd_reg)

centered_target = target_slope - np.nanmean(target_slope)
artificialPSF = pancake.analysis.klip_projection(centered_target,sgd_reg)

After registering the SGD observations to the target image, treat the SGD results as a reference library for KLIP and generate an artificial PSF for reference-subtraction

In [ ]:
sgd_sub = centered_target - artificialPSF

plt.figure(figsize=(15,4))
plt.subplot(131)
plt.imshow(centered_target)
plt.title('Target Slope Image')
plt.colorbar().set_label('e$^{-}$/s')
plt.subplot(132)
plt.imshow(artificialPSF)
plt.title('KLIP Artificial PSF')
plt.colorbar().set_label('e$^{-}$/s')
plt.subplot(133)
plt.imshow(sgd_sub)
plt.title('Artificial PSF Subtracted')
plt.colorbar().set_label('e$^{-}$/s')

# The Easy Way

You can obtain the same result as above by using the ```calculate_subtracted``` convenience function:

In [ ]:
sub_results = pancake.engine.calculate_subtracted(config, ta_error=True, sgd=True, stepsize=20.e-3)

plt.figure(figsize=(15,4))
plt.subplot(131)
plt.imshow(sub_results['target'])
plt.title('Target Slope Image')
plt.colorbar().set_label('e$^{-}$/s')
plt.subplot(132)
plt.imshow(sub_results['psf'])
plt.title('KLIP Artificial PSF')
plt.colorbar().set_label('e$^{-}$/s')
plt.subplot(133)
plt.imshow(sub_results['subtracted'])
plt.title('Artificial PSF Subtracted')
plt.colorbar().set_label('e$^{-}$/s')